### 관찰 및 아이디어
- 인접한 칸을 감시해야 한다길래, 한명의 간수가 최대 3명까지 볼 수 있는줄 알았다. 그냥 연속한 칸을 봐야한단 뜻이 맞을 것이다.
  - 추가로, 만약 이 조건이 없다면 C를 정렬한 뒤에 그리디하게, 혹은 간단한 DP로도 풀릴 것이다. 하지만 이 조건으로 인해 순서를 바꿔선 안된다.
- 모든 감옥칸은 감시되어야 한다는 내용이 빠져있는 것 같은데, 일단 그래야 하는 것이 당연하니 그런걸로 생각하면 될 것 같다.
- $L \leq G$ 이면 $\sum(L)$ 과 같다.
- 어떤 간수가 구간 $[i, j)$ 를 감시한다고 할 때, 그 간수로써 발생하는 탈옥 위험도는 $\displaystyle C'(i, j) = (j - i) \cdot \sum_{k= i \to j-1}(C_k)$ 이다.

### 풀이
- DP 설계를 못하고 끙끙대다가 결국 답지를 봤다. 
  - get optimal result at length $L[:j]$ on stage $i$ 꼴은 많이 봐왔던 설계인데 제대로 풀지 못한게 약간 아쉽다.
  - $\displaystyle DP[i][j] = \min_{0 \leq k \leq i}(DP[t-1][k] + C[k+1][i])$
  - 구간 $[i, k]$ 에 대해 감시 비용은 $\displaystyle C[k+1][i] = \sum_{l = k + 1 \to i}(C_l) \cdot (i - k)$ 이므로 이거는 맞다.
    - sum은 당연히 누적합을 사용해 $\Omicron(1)$ 에 구할 수 있다.
    - 구간합 배열이고 각 원소가 양의 정수이므로 $k$ 가 증가할 때마다 $C[i][k]$ 는 단조 증가한다. \
    $c < d$ 인 $c, d$ 에서 $C[i][c] \le C[i][d]$ 를 만족하므로 DnC Opt 조건을 만족한다.
    - 비슷하게 접근했고, 위 까지만 보여도 되는데 이상한 생각을 너무 많이해서 시간을 낭비했다.

In [ ]:
import io, os
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

def sol() :
  L, G = map(int, input().split())
  C = list(map(int, input().split()))
  acc = [0]
  for i, v in enumerate(C) :
    acc.append(acc[i] + v)
  calc = lambda i, j: (acc[j] - acc[i]) * (j - i) #구간 [i, j) 의 탈옥 위험도 기여량

  DP = [[0] * (L+1) for _ in range(G+1)] #DP[t][i] : t명의 간수가 i번째 죄수까지 감시할 때의 최소 탈옥 위험도
  P = [[0] * (L+1) for _ in range(G+1)]  #P[t][i] : pivot, DP[t][i]를 구할 때 마지막 간수가 감시할 죄수의 인덱스
  for i in range(L+1) :
    DP[1][i] = calc(0, i)
    P[1][i] = 0
  
  def DnC(t):
    S = [(0, L, 0, L)]
    while S:
      s, e, l, r = S.pop()
      if s > e: continue

      m = (s + e) // 2
      DP[t][m] = None
      P[t][m] = None

      for k in range(l, r + 1) :
        res = DP[t-1][k] + calc(k, m)
        if DP[t][m] == None or DP[t][m] > res : #최소값 및 pivot 갱신
          DP[t][m] = res
          P[t][m] = k
      S.append((s, m - 1, l, P[t][m]))
      S.append((m + 1, e, P[t][m], r))
  
  for t in range(2, G+1) :
    DnC(t)
  
  print(DP[G][L])

sol()

- 냅색 마냥 1차원으로 최적화할 수 있다.

In [ ]:
import io, os
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

NONE = -1
def sol() :
  L, G = map(int, input().split())
  C = list(map(int, input().split()))
  acc = [0]
  DP = [] #DP[i] : i번째 죄수까지 감시할 때의 최소 탈옥 위험도. 
  tmp = [0] * L #위 DP는 G번 수행되지만, 바로 이전의 정보만 있으면 되므로 tmp 같이 swap하면서 사용
  for i, v in enumerate(C) :
    s = acc[i] + v
    acc.append(s)
    DP.append(s * (i + 1))

  calc = lambda i, j: (acc[j] - acc[i+1]) * (j - i - 1) #구간 [i, j)에 대한 간수의 탈옥 위험도 기여량
  def DnC():
    S = [(0, L-1, 0, L-1)]
    while S:
      s, e, l, r = S.pop()
      if s > e: continue

      m = (s + e) // 2
      p = s #최소가 되게 하는 가장 왼쪽의 pivot
      for k in range(l, min(m, r) + 1) :
        if DP[p] + calc(p, m+1) > DP[k] + calc(k, m+1) : #pivot 갱신
          p = k
      
      tmp[m] = DP[p] + calc(p, m+1)
      S.append((s, m - 1, l, p))
      S.append((m + 1, e, p, r))
  
  for _ in range(G-1) :
    DnC()
    DP, tmp = tmp, DP
  
  print(DP[-1])

sol()